https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb#scrollTo=KuUbPW7oV-B5

- Wav2Vec2 was pretrained on the audio data of LibriSpeech and LibriVox which both were sampling with 16kHz
- https://github.com/pytorch/fairseq/blob/fcca32258c8e8bcc9f9890bf4714fa2f96b6b3e1/fairseq/models/wav2vec/wav2vec2_asr.py
- https://github.com/pytorch/fairseq/issues/2917
- https://github.com/vrindaprabhu/datasets/blob/186a657263ee07b8583cacd7f6b4a1082ae829d4/datasets/timit_asr/timit_asr.py

In [1]:
import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2

In [5]:
%env DATASET_PATH=dummy

env: DATASET_PATH=dummy


In [6]:
from data_utils import VCTKSanity
vctk = VCTKSanity()

FileNotFoundError: [Errno 2] No such file or directory: 'data/vctk_sanity.pkl'

In [ ]:
%%capture output
device="cuda"
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "facebook/wav2vec2-base"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

In [ ]:
from data_utils import VCTK

data_collator = DataCollator(processor=processor)
vctk = VCTK("/shared/g-luo/vctk")
vctk = vctk.map(lambda batch: preprocess(batch, processor))

In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir="./runs",
#   group_by_length=True,
#   per_device_train_batch_size=32,
#   evaluation_strategy="steps",
#   num_train_epochs=30,
#   fp16=True,
#   gradient_checkpointing=True,
#   save_steps=500,
#   eval_steps=500,
#   logging_steps=500,
#   learning_rate=1e-4,
#   weight_decay=0.005,
#   warmup_steps=1000,
#   save_total_limit=2,
#   push_to_hub=False,
# )

In [ ]:
# from transformers import Trainer
# from torch.utils.data import Subset

# trainer = Trainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=lambda pred: compute_metrics(pred, processor, wer_metric),
#     train_dataset=train_dataset,
#     eval_dataset=train_dataset,
#     tokenizer=processor.feature_extractor,
# )

Scratchbook

In [ ]:
from data_utils import VCTK

DATASET_PATH="/shared/g-luo/vctk"
vctk = VCTK(path=DATASET_PATH, sample_rate=16000)

audio = vctk.__getitem__(0)["waveform"]
rate = 16000
input_values = processor(audio[0], sampling_rate = rate, return_tensors = 'pt').input_values
input_values = input_values.to(device)

In [ ]:
import torch
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim =-1)
transcriptions = tokenizer.decode(predicted_ids[0])
print(transcriptions)
print(vctk.__getitem__(0)["text"])

In [ ]:
# import IPython.display as ipd
# import numpy as np
# import random

# ipd.Audio(data=np.asarray(), autoplay=True, rate=16000)